In [ ]:
import os

import numpy as np
import pandas as pd
import requests
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms import functional as F
from tqdm.notebook import tqdm
from ultralytics import YOLO

from datasets import Dataset as HF_Dataset
from datasets import Features, Value, concatenate_datasets, load_dataset
from utils import resize_preserve_aspect_ratio

In [ ]:
def get_num_people(result):
    if bool(result):
        labels_dict = result.names
        det_labels = [int(i.item()) for i in result.boxes.cls]
        det_labels = [labels_dict[i] for i in det_labels]
        num_people = len(det_labels)
        return num_people
    else:
        return 0

In [ ]:
def pad_to_fixed_size(img, size=(640, 640)):
    width, height = img.size
    # Calculate padding
    left = (size[0] - width) // 2
    top = (size[1] - height) // 2
    right = size[0] - width - left
    bottom = size[1] - height - top

    # Apply padding
    img_padded = F.pad(img, padding=(left, top, right, bottom))
    return img_padded

In [ ]:
dataset_id = "jordandavis/fashion"
ds = load_dataset(dataset_id, split="train", trust_remote_code=True, num_proc=os.cpu_count())

In [ ]:
class PinterestDataset(Dataset):
    def __init__(self, dataset_id=None, image_col="image", image_id_col=None):
        self.ds = load_dataset(dataset_id, split="train", trust_remote_code=True, num_proc=os.cpu_count())
        self.image_col = image_col
        self.image_id_col = image_id_col
        self.imgsz = 640
        self.half = False
        self.transform = transforms.Compose([
            transforms.Lambda(lambda img: pad_to_fixed_size(img, (640, 640))),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        image_pil = item[self.image_col]

        if image_pil.mode != "RGB":
            image_pil = image_pil.convert("RGB")

        image = resize_preserve_aspect_ratio(image_pil, self.imgsz)
        image = self.transform(image).unsqueeze(0)

        image_id = item.get(self.image_id_col, idx)

        return {"image_id": image_id, "image": image}

In [ ]:
# Load the YOLO Model
path = "weights/yolov8n-seg.pt"
model = YOLO(path, verbose=False)

In [ ]:
model.compile()

In [ ]:
# Enter the dataset ID and load it as a torch dataset
dataset_id = "jordandavis/fashion"
ds = PinterestDataset(dataset_id=dataset_id, image_col="image", image_id_col=None)

In [ ]:
# Dataloader


def collate_fn(ex):
    images = torch.cat([e["image"] for e in ex], dim=0)
    image_ids = [e["image_id"] for e in ex]
    return dict(images=images, image_ids=image_ids)


workers = os.cpu_count()
batch_size = 256
dataloader = DataLoader(
    ds,
    collate_fn=collate_fn,
    batch_size=batch_size,
    num_workers=16,
    pin_memory=True,
    shuffle=False,
)

In [ ]:
num_people_results = {}

In [ ]:
max_people = 0
with tqdm(total=len(dataloader)) as pbar:
    for batch in dataloader:
        image_ids = batch.get("image_ids")
        with torch.no_grad():
            images = batch.get("images").to("cuda")
            results = model(images, classes=0, verbose=False)

        num_people = [get_num_people(result) for result in results]
        max_people_batch = max(num_people)
        if max_people_batch > max_people:
            max_people = max_people_batch
            print(f"Max people detected: {max_people}", end="\r")

        result = dict(zip(image_ids, num_people))
        num_people_results.update(result)
        pbar.update(1)

import json

with open("num_people_results.json", "w") as f:
    f.write(json.dumps(num_people_results))

In [ ]:
# Load the results into a pandas dataframe
df = pd.DataFrame(num_people_results.items(), columns=["image_id_processed", "num_people"])

# Convert that to a HF Dataset
ds_people = HF_Dataset.from_pandas(df[['num_people']])

# Concatenate the two datasets
new_ds = concatenate_datasets([ds.ds, ds_people], axis=1)

In [ ]:
new_ds = new_ds.cast_column("width", Value('int64'))
new_ds = new_ds.cast_column("height", Value('int64'))
new_ds = new_ds.cast_column("id", Value('int64'))

In [ ]:
from huggingface_hub import create_repo, upload_folder

repo_id = "jordandavis/fashion_num_people"

create_repo(
    repo_id=repo_id,
    repo_type="dataset",
    exist_ok=True,
)

In [ ]:
commit_description = "Add the num_people column"
new_ds.push_to_hub(repo_id, commit_description=commit_description)